In [1]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip
  
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2021-05-26 09:09:42--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.80, 172.217.15.80, 172.217.12.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   227MB/s    in 0.3s    

2021-05-26 09:09:43 (227 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [3]:
train_gen = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest').flow_from_directory(
        train_dir,
        target_size=(150,150),
        batch_size=20,
        class_mode='binary')
validation_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
        validation_dir,
        target_size=(150,150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [5]:
history = model.fit(
      train_gen,
      steps_per_epoch=10,  # 2000 images = batch_size * steps
      epochs=100,
      validation_data=validation_gen,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

Epoch 1/100
10/10 - 36s - loss: 0.7100 - accuracy: 0.4900 - val_loss: 0.6944 - val_accuracy: 0.5000
Epoch 2/100
10/10 - 4s - loss: 0.6977 - accuracy: 0.5150 - val_loss: 0.6954 - val_accuracy: 0.5000
Epoch 3/100
10/10 - 4s - loss: 0.6906 - accuracy: 0.5050 - val_loss: 0.7109 - val_accuracy: 0.5000
Epoch 4/100
10/10 - 4s - loss: 0.7004 - accuracy: 0.4950 - val_loss: 0.6929 - val_accuracy: 0.4990
Epoch 5/100
10/10 - 4s - loss: 0.6936 - accuracy: 0.4800 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 6/100
10/10 - 4s - loss: 0.6948 - accuracy: 0.4400 - val_loss: 0.6897 - val_accuracy: 0.5920
Epoch 7/100
10/10 - 4s - loss: 0.6942 - accuracy: 0.5600 - val_loss: 0.6895 - val_accuracy: 0.6170
Epoch 8/100
10/10 - 4s - loss: 0.6912 - accuracy: 0.5300 - val_loss: 0.6877 - val_accuracy: 0.5720
Epoch 9/100
10/10 - 4s - loss: 0.6936 - accuracy: 0.5100 - val_loss: 0.6869 - val_accuracy: 0.6040
Epoch 10/100
10/10 - 4s - loss: 0.6823 - accuracy: 0.6000 - val_loss: 0.7073 - val_accuracy: 0.5000
Epoch 11

In [6]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

--2021-05-26 09:18:03--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.7.240, 172.217.15.80, 172.217.13.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.7.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   224MB/s    in 0.4s    

2021-05-26 09:18:04 (224 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

last layer output shape:  (None, 7, 7, 768)


In [7]:

from tensorflow.keras import layers
from tensorflow.keras import Model
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense  (1, activation='sigmoid')(x)           

model2 = Model( pre_trained_model.input, x) 

model2.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [8]:
history2 = model2.fit(
      train_gen,
      steps_per_epoch=10,  # 2000 images = batch_size * steps
      epochs=100,
      validation_data=validation_gen,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

Epoch 1/100
10/10 - 8s - loss: 0.8789 - accuracy: 0.6750 - val_loss: 0.2083 - val_accuracy: 0.9220
Epoch 2/100
10/10 - 4s - loss: 0.2783 - accuracy: 0.9000 - val_loss: 0.1913 - val_accuracy: 0.9120
Epoch 3/100
10/10 - 4s - loss: 0.3521 - accuracy: 0.8350 - val_loss: 0.1280 - val_accuracy: 0.9480
Epoch 4/100
10/10 - 4s - loss: 0.2439 - accuracy: 0.9000 - val_loss: 0.1209 - val_accuracy: 0.9530
Epoch 5/100
10/10 - 4s - loss: 0.2830 - accuracy: 0.8800 - val_loss: 0.1792 - val_accuracy: 0.9260
Epoch 6/100
10/10 - 4s - loss: 0.2096 - accuracy: 0.9150 - val_loss: 0.1096 - val_accuracy: 0.9540
Epoch 7/100
10/10 - 4s - loss: 0.2056 - accuracy: 0.9150 - val_loss: 0.1521 - val_accuracy: 0.9340
Epoch 8/100
10/10 - 4s - loss: 0.2243 - accuracy: 0.8950 - val_loss: 0.1257 - val_accuracy: 0.9440
Epoch 9/100
10/10 - 4s - loss: 0.1808 - accuracy: 0.9500 - val_loss: 0.1157 - val_accuracy: 0.9490
Epoch 10/100
10/10 - 4s - loss: 0.2948 - accuracy: 0.8850 - val_loss: 0.1458 - val_accuracy: 0.9460
Epoch 11/

In [9]:
import numpy as np

image = tf.keras.preprocessing.image.load_img(path='/content/istockphoto-1045886560-612x612.jpg', target_size=(150,150))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  
predictions = model.predict(input_arr)
predictions2 = model2.predict(input_arr)


In [10]:
print(predictions, predictions2)

[[1.]] [[0.]]


In [11]:
train_gen.class_indices

{'cats': 0, 'dogs': 1}